<a href="https://colab.research.google.com/github/Coralain10/ML_Archaeological-Objects-Reconstruction/blob/main/KerasSimpleModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import math

In [58]:
!gdown 1AIQPe2G6ZHPwZLmo19-ewbqCNT7TDfYE

Downloading...
From: https://drive.google.com/uc?id=1AIQPe2G6ZHPwZLmo19-ewbqCNT7TDfYE
To: /content/FullAndFracture.npy
100% 466M/466M [00:02<00:00, 168MB/s]


In [59]:
!gdown 1U589karMjF_k4nsWe5Rb8SXTp1YajhN9

Downloading...
From: https://drive.google.com/uc?id=1U589karMjF_k4nsWe5Rb8SXTp1YajhN9
To: /content/model.zip
100% 466M/466M [00:05<00:00, 81.4MB/s]


In [60]:
!unzip model.zip

Archive:  model.zip
replace content/saved_model/keras_metadata.pb? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [61]:
!gdown 1izkjM3BLA5_WYrImCSZ3RpH8Th1NQfkp

Downloading...
From: https://drive.google.com/uc?id=1izkjM3BLA5_WYrImCSZ3RpH8Th1NQfkp
To: /content/broken_chair.npy
100% 32.9k/32.9k [00:00<00:00, 18.1MB/s]


In [62]:
data = np.load('FullAndFracture.npy', allow_pickle=True)

In [63]:
data = np.array(data, dtype = np.int32)

In [64]:
X = data[1]
Y = data[0]

In [65]:
lx = X.shape[0]
ly = Y.shape[0]

In [66]:
X = list(X)
Y = list(Y)

In [67]:
for i in range(lx):
  X[i] = X[i].flatten()
for i in range(ly):
  Y[i] = Y[i].flatten()

In [68]:
X, Y = np.array(X), np.array(Y)

In [69]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, train_size=0.7)

In [70]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(4978, 32768) (4978, 32768)
(2134, 32768) (2134, 32768)


In [71]:
model = tf.keras.models.load_model('/content/content/saved_model')

In [72]:
prediction = model.predict(x_val)

67/67 [==============================] - 7s 100ms/step


In [73]:
prediction = np.around(prediction)
prediction = np.array(prediction, dtype = np.int32)

In [74]:
patches = []
for i in range(10):
  patch = []
  for j in range(32768):
    if prediction[i][j] == x_val[i][j]:
      patch.append(0)
    else:
      patch.append(1)
  patches.append(patch)
patches = np.array(patches)

In [75]:
def plot3d(verts, s=10, c=(105,127,155), show_grid=False):
    x, y, z = zip(*verts)
    color = f'rgb({c[0]}, {c[1]}, {c[2]})'
    trace = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=s,
            color=color,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=1
        )
    )
    data = [trace]
    layout = go.Layout(
        margin=dict(l=0, r=0, b=0, t=0),
        scene = go.Scene(
            xaxis=dict(visible=show_grid),
            yaxis=dict(visible=show_grid),
            zaxis=dict(visible=show_grid)
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [76]:
def point_cloud_to_volume(points, vsize, radius=1.0):
    vol = np.zeros((vsize,vsize,vsize), dtype=np.bool)
    voxel = 2*radius/float(vsize)
    locations = (points + radius)/voxel
    locations = locations.astype(int)
    vol[locations[:,0],locations[:,1],locations[:,2]] = 1.0
    
    return vol

In [77]:
def volume_to_point_cloud(vol):
    """ vol is occupancy grid (value = 0 or 1) of size vsize*vsize*vsize
        return Nx3 numpy array.
    """
    vsize = vol.shape[0]
    assert(vol.shape[1] == vsize and vol.shape[1] == vsize)
    points = []
    for a in range(vsize):
        for b in range(vsize):
            for c in range(vsize):
                if vol[a,b,c] == 1:
                    points.append(np.array([a,b,c]))
    if len(points) == 0:
        return np.zeros((0,3))
    points = np.vstack(points)
    
    return points

def auto_pcl_to_volume(points, vsize):
	data_min = np.min(points)
	data_max = np.max(points)
	radius = max(abs(data_min), data_max)
	radius = math.ceil(radius*100) / 100
	vol = point_cloud_to_volume(points, vsize, radius)

	return vol

In [78]:
def plot_vol(vol, s=10, c=(105,127,155), show_grid=False):
    if vol.dtype != np.bool:
        vol = vol > 0

    pc = volume_to_point_cloud(vol)
    plot3d(pc, s, c, show_grid)

In [79]:
def plot_reconstruction(vol1, vol2, s=10,
                        c2=(105,128,155), c1=(182,49,62), show_grid=False):
    if vol1.dtype != np.bool:
        vol1 = vol1 > 0
    if vol2.dtype != np.bool:
        vol2 = vol2 > 0
        
    color1 = f'rgb({c1[0]}, {c1[1]}, {c1[2]})'
    color2 = f'rgb({c2[0]}, {c2[1]}, {c2[2]})'
    vol2 = np.logical_xor(vol2, vol1)
    pc1 = volume_to_point_cloud(vol1)
    pc2 = volume_to_point_cloud(vol2)
    x1, y1, z1 = zip(*pc1)
    x2, y2, z2 = zip(*pc2)
    trace1 = go.Scatter3d(
        x=x1, y=y1, z=z1,
        mode='markers',
        marker=dict(
            size=s,
            color=color1,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=1
        )
    )
    trace2 = go.Scatter3d(
        x=x2, y=y2, z=z2,
        mode='markers',
        marker=dict(
            size=s,
            color=color2,
            line=dict(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5
            ),
            opacity=1
        )
    )
    data = [trace1, trace2]
    layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0), 
                       scene = go.Scene(
                               xaxis=dict(visible=show_grid),
                               yaxis=dict(visible=show_grid),
                               zaxis=dict(visible=show_grid)),
                       showlegend=False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [80]:
x_val = np.reshape(x_val, (2134, 32, 32, 32))
prediction = np.reshape(prediction, (2134, 32, 32, 32))

In [81]:
plot_vol(x_val[200])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [82]:
plot_reconstruction(prediction[200] - x_val[200], prediction[200N], s = 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [83]:
broken_chair = np.load('/content/broken_chair.npy')

In [84]:
broken_chair = np.reshape(broken_chair.flatten(), (1, 32*32*32))

In [85]:
bcpred = model.predict(broken_chair)

1/1 [==============================] - 0s 80ms/step


In [86]:
broken_chair = np.reshape(broken_chair, (32,32,32))
bcpred = np.around(np.reshape(bcpred, (32,32,32)))

In [87]:
plot_reconstruction(bcpred - broken_chair, bcpred, s = 10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning:

`np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [88]:
model.save('saved_model')